In [6]:
from pathlib import Path
import xmltodict

summary_path = Path("./ORCID_2023_10_activities_0")

In [8]:
all_subfolders = summary_path.iterdir()

authors_with_works_counter = 0
total_works_counter = 0

for subfolder in all_subfolders:
    # e.g. a subfolder in subfolders: ORCID_2023_10_activities_0/510

    for orcid_folder in subfolder.iterdir():
        # e.g. orcid_folder = ORCID_2023_10_activities_0/380/0000-0002-5250-5380

        for activity_folder in orcid_folder.iterdir():
            # e.g. activity_folder = ORCID_2023_10_activities_0/510/0000-0003-4496-9510/works

            if activity_folder.name == "works":
                authors_with_works_counter += 1
                works_files = activity_folder.iterdir()
                for work in works_files:
                    with open(work, "r") as xml_file:
                        xml_string = xml_file.read()
                        try:
                            python_dict = xmltodict.parse(xml_string)
                        except Exception as e:
                            print(f"An error occurred while parsing file {work}, and here it is: ")
                            print(e)

                            # Skip if there was an error parsing. Nothing that can be done, the file is
                            # probably corrupt.

                            continue
                    total_works_counter += 1

                break

print("All authors with works count: ", authors_with_works_counter)
print("Total amount of works: ", total_works_counter)

An error occurred while parsing file ORCID_2023_10_activities_0/090/0000-0002-8045-4090/works/0000-0002-8045-4090_works_130455429.xml, and here it is: 
no element found: line 2, column 0
An error occurred while parsing file ORCID_2023_10_activities_0/930/0000-0002-5249-3930/works/0000-0002-5249-3930_works_72412146.xml, and here it is: 
no element found: line 2, column 0
An error occurred while parsing file ORCID_2023_10_activities_0/630/0000-0002-8241-0630/works/0000-0002-8241-0630_works_130623121.xml, and here it is: 
no element found: line 2, column 0
An error occurred while parsing file ORCID_2023_10_activities_0/380/0000-0003-4320-4380/works/0000-0003-4320-4380_works_129788586.xml, and here it is: 
no element found: line 2, column 0
An error occurred while parsing file ORCID_2023_10_activities_0/170/0000-0003-1010-7170/works/0000-0003-1010-7170_works_130453272.xml, and here it is: 
no element found: line 2, column 0
An error occurred while parsing file ORCID_2023_10_activities_0/62

In [1]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)
db = client.final_orcid_database

In [4]:
results = db.works.aggregate([
    { "$group": { "_id": "$put_code", "count": { "$sum": 1 } } },
    { "$match": { "_id": { "$ne": None }, "count": { "$gt": 1 } } },
    { "$project": { "name": "$_id", "_id": 0 } }
])

In [2]:
db.works.aggregate([
    {
        "$group": {
            "_id": "$put_code",
            "doc": { "$first": "$$ROOT" },
        }
    },
    {
        "$replaceWith": "$doc"
    },
    {
        "$out": "works_without_duplicates"
    }
])